In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
import os
import glob
import random

In [2]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Bidirectional,ConvLSTM2D,BatchNormalization,Conv3D
from keras import Input
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,SimpleRNN,GRU
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

In [3]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
def generate_csv(folder, labels):
    folder_name = os.path.basename(folder)
    print(labels)
    # generate CSV file
    df = pd.DataFrame(columns=["filepath", "label"])
    i = 0
    for label in labels:
      print("Reading", os.path.join(folder, label))
      for filepath in glob.glob(os.path.join(folder, label,"*.wav")):
        df.loc[i] = [filepath,label]
        i += 1
    output_file = f"{folder_name}.csv"
    print("Saving", output_file)
    df.to_csv(output_file)

dir_folder = '/content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data'
labels = ['2_Batdieuhoa','3_Tatdieuhoa','4_Tang1do','7_Giam1do','16_Bat26do']
generate_csv(dir_folder,labels)
metadata_filename = "Data.csv"

['2_Batdieuhoa', '3_Tatdieuhoa', '4_Tang1do', '7_Giam1do', '16_Bat26do']
Reading /content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/2_Batdieuhoa
Reading /content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/3_Tatdieuhoa
Reading /content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/4_Tang1do
Reading /content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/7_Giam1do
Reading /content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/16_Bat26do
Saving Data.csv


In [12]:



# load CSV files as DataFrames
metadata = pd.read_csv(metadata_filename)
metadata.tail()

,Unnamed: 0,filepath,label
1291,1291,/content/gdrive/MyDrive/DATN/Augument/150s_28_...,16_Bat26do
1292,1292,/content/gdrive/MyDrive/DATN/Augument/150s_28_...,16_Bat26do
1293,1293,/content/gdrive/MyDrive/DATN/Augument/150s_28_...,16_Bat26do
1294,1294,/content/gdrive/MyDrive/DATN/Augument/150s_28_...,16_Bat26do
1295,1295,/content/gdrive/MyDrive/DATN/Augument/150s_28_...,16_Bat26do


In [13]:
!pip install audiomentations

In [14]:
from typing import List
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Clip, Padding, Reverse, Trim, BandPassFilter,Limiter
from audiomentations import BandStopFilter,Gain,TimeMask,HighPassFilter,ClippingDistortion,ClippingDistortion,GainTransition,LowPassFilter,AirAbsorption


def get_augs(aug):
    augmentations = []
    if aug == 'NoiseInjection':
        augmentations.append(AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.0014, p=1.0))
    elif aug == 'PitchShifting':
        augmentations.append(PitchShift(min_semitones=-2.5, max_semitones=2.5, p=1.0))
    elif aug == 'TimeStretching':
        augmentations.append(TimeStretch(min_rate=1.04, max_rate=1.05, p=1.0))
    elif aug == 'Padding':
        augmentations.append(Padding(min_fraction=0.04,max_fraction=0.05, p=1.0))
    elif aug == 'BandPassFilter':
        augmentations.append(BandPassFilter(min_center_freq=2200, max_center_freq=2400, p=1.0))
    elif aug == 'Gain':
      augmentations.append(Gain(min_gain_in_db=-10, max_gain_in_db=10,p=1.0))
    elif aug == 'TimeMask':
      augmentations.append(TimeMask(min_band_part=0.14, max_band_part=0.16,p=1.0))
    elif aug == 'HighPassFilter':
      augmentations.append(HighPassFilter(min_cutoff_freq=1600, max_cutoff_freq=1800, p=1))
    elif aug == 'ClippingDistortion':
      augmentations.append(ClippingDistortion(min_percentile_threshold =0, max_percentile_threshold=8, p=1))
    elif aug == 'AirAbsorption':
      augmentations.append(AirAbsorption(min_distance=10.0,max_distance=40.0,p=1.0,))
    elif aug == "Trim":
      augmentations.append(Trim(top_db=30.0,p=1.0))
    return augmentations

In [15]:
import soundfile as sf
import shutil
augs = ['NoiseInjection','PitchShifting','TimeStretching','Padding','BandPassFilter','Gain',
        'TimeMask','HighPassFilter','ClippingDistortion','AirAbsorption','Trim']
from tqdm import tqdm
i=0
for index_num,row in tqdm(metadata.iterrows()):
    filename = str(row["filepath"])
    label=row["label"]
    folder = '/content/gdrive/MyDrive/DATN/Augument/150s_28_01/Data/'+label
    os.makedirs(folder, exist_ok = True)

    for aug in augs:

      y, sr = librosa.load(filename,sr=16000)
      augment = Compose(get_augs(aug=aug))
      wav = augment(samples=y, sample_rate=sr)
      sf.write(aug+'_'+'part'+str(i)+'.wav', wav, 16000, subtype='PCM_16')
      duong_dan_tep = os.path.join(folder, aug+'_'+'part'+str(i)+'.wav')
      if not (os.path.isfile(duong_dan_tep)):
        shutil.move(aug+'_'+'part'+str(i)+'.wav', folder)

      # duong_dan_tep = os.path.join(folder, aug+'_'+'part'+str(i)+'.wav')
      # if os.path.isfile(duong_dan_tep):
      #   os.remove(duong_dan_tep)

      i+=1
print(i)

1296it [11:35,  1.86it/s]

14256
